In [145]:
"""This notebook is a tool to predct the number of homeruns a player will hit in a season"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as ln


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [146]:
batting_stats_df = pd.read_csv('./data/batting-stats-9-21-2018.csv')
batted_ball_df = pd.read_csv('./data/batted-ball-9-21-2018.csv')
plate_discipline_df = pd.read_csv('./data/plate-discipline-9-21-2018.csv')

In [147]:
df = pd.merge(batting_stats_df.loc[:, ['Name', 'PA', 'HR']], batted_ball_df.loc[:, ['Name', 'Hard%', 'FB%']], left_on='Name', right_on='Name')
df = pd.merge(df, plate_discipline_df.loc[:, ['Name', 'Swing%', 'Contact%']], left_on='Name', right_on='Name')

In [148]:
df

,Name,PA,HR,Hard%,FB%,Swing%,Contact%
0,Mookie Betts,593.0,30.0,44.7 %,44.9 %,35.9 %,85.8 %
1,Mike Trout,578.0,36.0,44.5 %,44.2 %,37.3 %,84.3 %
2,Jose Ramirez,656.0,38.0,37.4 %,45.5 %,38.8 %,87.6 %
3,Francisco Lindor,699.0,36.0,41.1 %,39.2 %,46.8 %,83.6 %
4,Alex Bregman,669.0,30.0,36.3 %,43.8 %,37.3 %,88.4 %
5,Matt Chapman,582.0,23.0,43.3 %,40.5 %,41.7 %,79.1 %
6,Christian Yelich,605.0,31.0,47.4 %,22.5 %,45.1 %,78.5 %
7,Manny Machado,666.0,35.0,37.8 %,43.1 %,48.6 %,79.8 %
8,Anthony Rendon,559.0,22.0,38.6 %,42.5 %,47.4 %,87.4 %
9,Lorenzo Cain,581.0,10.0,39.1 %,23.3 %,42.0 %,83.5 %


In [149]:
df = df.dropna()

In [150]:
for col in df.columns:
    if '%' in col:
        df[col] = df[col].str.rstrip('%').astype('float') / 100

/home/chris/.venvs/baseball-data/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [151]:
X = df.loc[:, ['PA', 'Hard%', 'FB%', 'Swing%', 'Hard%']]
y = df.loc[:, ['HR']]
X = X.values.reshape(len(X), 5)
y = y.values.reshape(len(y), 1)

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [153]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [154]:
y_pred = regressor.predict(X_test)
y_pred.shape, y_test.shape

((29, 1), (29, 1))

In [155]:
res_df = pd.DataFrame({'Actual': y_test[:, 0], 'Predicted': y_pred[:, 0]})
res_df

,Actual,Predicted
0,18.0,18.066546
1,6.0,12.667206
2,35.0,36.252811
3,18.0,14.770232
4,33.0,31.784046
5,35.0,28.664421
6,15.0,15.029719
7,10.0,8.424417
8,19.0,22.509692
9,17.0,23.193694


In [156]:
print('Mean Absolute Error: {}'.format(metrics.mean_absolute_error(y_test, y_pred)))
print('Mean Squared Error: {}'.format(metrics.mean_squared_error(y_test, y_pred)))
print('Root Mean Squared Error {}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

Mean Absolute Error: 3.5873140194398494
Mean Squared Error: 19.285327855963683
Root Mean Squared Error 4.391506331085461
